In [ ]:
import tensorflow as tf
from tensorflow import keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.image as mpimg
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Lambda, LayerNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, History, EarlyStopping

In [1]:
from sample.functionImplemented import get_model, custom_loss, get_threshold, custom_scaler, get_opt_action, update_replay_helper, populate_replay_memory, update_replay_memory 
from sample.car import Car
from sample.track import Track

In [6]:
help(Car)

Help on class Car in module sample.car:

class Car(builtins.object)
 |  The car object has the following attributes:
 |  1. Track - the associated track
 |  2. spawn_at: Possible spawning indices
 |  3. current position: Current position on the track. Floating point coordinate.
 |  4. integer_position_: defined as current_position.astype('int16')
 |  5. Bool collided_on_last
 |  6. sensors: LIDAR type function
 |  7. init_speed_
 |  8. max_speed
 |  9. sampling frequency: The frequency of play.
 |  10. speed: Current speed
 |  11. car_angle: current car angle as define below.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, track, max_speed, spawn_at)
 |      track: The track as a binary array
 |      max_speed: Max attainable speed
 |      spawn_at: The possible spawning positions for the car. One among this is chosen.
 |      
 |      car angle is defined as the angle from the angle from
 |      the standard mathematical X axis, i.e., 
 |      y ^
 |        |
 |        |
 |    

In [ ]:
img = mpimg.imread("track_pic9.jpg")[:,:,0]
track1=(img<150).astype('int')
print(track1.shape)
track_rows, track_cols=track1.shape
pos_pos=np.where(track1==1)
spawning_positions=np.zeros((len(pos_pos[0]), 2))
spawning_positions[:, 0]=pos_pos[0]
spawning_positions[:, 1]=pos_pos[1]
spawning_positions=spawning_positions.astype('int')

track=Track(track1, spawning_positions)
plt.imshow(track1)

In [ ]:
actions=np.asarray([-1, 0, 1]) 
data_scaler=np.asarray([
    100, 100, 100, 100,
    100, 100, 100, 100,
    50, 1, 1, 1
])
usescaler=True
trainedModel=tf.keras.models.load_model("TrainedModels/trainedModel1.h5", custom_objects={'custom_loss':custom_loss})

In [ ]:
new_car=Car(track.track, 50, track.spawn_at)
new_car.sampling_frequency=20


def get_plot(positions, superimposeon_this):
    x, y=positions
    for x_diff in range(-30, 31):
        for y_diff in range(-30, 31):
            if np.sqrt(x_diff**2+y_diff**2)<14:
                superimposeon_this[x+x_diff][y+y_diff]=1
    f=plt.figure(figsize=(10, 20))
    plt.imshow(superimposeon_this+new_car.track)
    plt.show()
    return


base_fig=np.zeros((track_rows, track_cols))


for iteration in range(3000):
    r, c=new_car.integer_position_
    for x_diff in range(-7, 8):
        for y_diff in range(-7, 8):
            if np.sqrt(x_diff**2+y_diff**2)<4:
                if r+x_diff<new_car.track.shape[0] and c+y_diff<new_car.track.shape[1]:
                    base_fig[r+x_diff][c+y_diff]=1
                    
                    
    direction, best_action_value=get_opt_action(new_car, trainedModel, actions, data_scaler, usescaler)
    print(direction, new_car.sensors[4], new_car.speed)
    
    theta=new_car.car_angle
    if direction==-1:
        theta=theta+np.pi/2
    if direction==1:
        theta=theta-np.pi/2
        
    f1, f2=np.sin(theta), np.cos(theta)
    new_car.execute_forces(f1, f2, max_magnitudes=500)
    
    if new_car.collided_on_last:
        print("boom")
        break
        
get_plot(new_car.integer_position_, base_fig)  

In [ ]:
plt.imshow(base_fig+track1)